# Prep Website Data

Prep datafiles for use in specific sections of the website


In [2]:
import numpy as np
import pandas as pd
import pydeck as pdk
import os
from os.path import join
import matplotlib.pyplot as plt
import seaborn as sns

import random
import json

In [29]:
dataDir = '../data'
outputDir = '../websiteData'

In [3]:
arts = []
with open(join(dataDir, 'articleMetadata.json')) as f:
    for i,line in enumerate(f):
        thisArt = json.loads(line)
        if 'isValid' in thisArt.keys():
            arts.append(thisArt)

## Hero

Create a sample of article titles used to drive the hero animation. This sample should be manually spot checked afterward to prune out titles that are less obviously related to the current pandemic. 

In [25]:
# get a random sample of journal articles
random.seed(101101)
sampleArts = random.sample(arts, 1000)

# extract the relevant metadata (title, journal name, etc) for each
# and save to csv
heroData = []
for a in sampleArts:
    title = a['title']
    if title:
        title = title.replace('\n', '').strip()
        
    journal = a['journalTitle']
    if journal:
        journal = journal.replace('\n', '').strip()
    
    heroData.append({
        'title': title,
        'journal': journal,
        'pubDate': a['pubDate']
    })

heroData_df = pd.DataFrame(heroData)
heroData_df.to_csv(join(outputDir, 'heroData.csv'), index=False)

In [22]:
heroData_df.shape

(1000, 3)

## Section I

### Papers per year

For the visualization showing the number of coronavirus related papers from the late 60's on to 2020. 

No processing steps required here, just a note about how that data was acquired: 

The `papersPerYear.csv` was obtained by going to pubmed central and using the covid-19 search string, without the publication date range:

```
"COVID-19"[All Fields] OR "COVID-19"[MeSH Terms] OR "COVID-19 Vaccines"[All Fields] OR "COVID-19 Vaccines"[MeSH Terms] OR "COVID-19 serotherapy"[All Fields] OR "COVID-19 Nucleic Acid Testing"[All Fields] OR "covid-19 nucleic acid testing"[MeSH Terms] OR "COVID-19 Serological Testing"[All Fields] OR "covid-19 serological testing"[MeSH Terms] OR "COVID-19 Testing"[All Fields] OR "covid-19 testing"[MeSH Terms] OR "SARS-CoV-2"[All Fields] OR "sars-cov-2"[MeSH Terms] OR "Severe Acute Respiratory Syndrome Coronavirus 2"[All Fields] OR "NCOV"[All Fields] OR "2019 NCOV"[All Fields] OR (("coronavirus"[MeSH Terms] OR "coronavirus"[All Fields] OR "COV"[All Fields]) ) 
```

Next, using the returned results, I applied a custom filter (using the left hand panel) to define a date range for each year from 1965 onwards. For instance, 1965 was setting the custom date range to `1965/01/01` to `1965/12/31`. 

I manually recorded the number of articles returned by this filter. Repeated for every year thereafter. 



## Section II

### Collaboration map

In order to store this data as efficiently as possible, create separate datasets for:

* a main geoIDs dataset. One row per unique geoID, stores all of the relevant info (addr, state, country, lat, lng) for each geoID. This dataset should be easily indexable

* the collaborations. For each collaboration, we need to know (at minimum) the date, src geoID idx, dst geoID idx

* total article states by date. For each date throughout 2020, store the number of (authors? papers? collaborations?) accumulated by that date



In [16]:
### Create the main geoIDs dataset
cityGeocodes_df = pd.read_csv(join(dataDir, 'cityGeocodes.csv'))

# add idx column
cityGeocodes_df['idx'] = cityGeocodes_df.index

# save a version for site, removing all unecessary columns
siteGeoIDs = cityGeocodes_df[['idx', 'lat', 'lng']]

# write to web folder
siteGeoIDs.to_csv(join(outputDir, 'geoIDs.csv'), float_format='%.2f', index=False)

In [21]:
def getGeoIdx(geoID):
    """ return the index of the specified geoID from the cityGeoCodes_df dataframe """
    indices = cityGeocodes_df.index[cityGeocodes_df['geoID'] == geoID].tolist()
    return indices[0]  # should only ever be 1 value

This step takes ~10min to run

In [42]:
def getGeoIdx(geoID):
    """ return the index of the specified geoID from the cityGeoCodes_df dataframe """
    indices = cityGeocodes_df.index[cityGeocodes_df['geoID'] == geoID].tolist()
    return indices[0]  # should only ever be 1 value


### create the collaborations dataset
collabs_df = pd.read_csv(join(dataDir, 'processed/collaborations.csv'))

# add the srcIdx and dstIdx cols to each row
collabs_df['srcIdx'] = collabs_df['geoID_A'].apply(getGeoIdx)
collabs_df['dstIdx'] = collabs_df['geoID_B'].apply(getGeoIdx)

# drop missing rows
collabs_df.dropna(inplace=True)

# drop the year from pub date to save space
collabs_df['pubDate'] = collabs_df['pubDate'].apply(lambda x: '-'.join(x.split('-')[1:]))

# save a version for site, removing unecessary columns
siteCollabs = collabs_df[['pubDate', 'srcIdx', 'dstIdx']].sort_values('pubDate')


In [44]:
# write data to file
siteCollabs.to_csv(join(outputDir, 'collabsByDate.csv'), index=False)

In [36]:
'-'.join(d.split('-')[1:])

'01-01'

In [10]:
cityGeocodes_df.index[cityGeocodes_df['geoID'] == 'ChIJU3OqqNAywkcRUG1NL6uZAAQ'].tolist()

[8039]